In this post, we will read multiple `.csv` files into Tensorflow using generators. But the method we will discuss is general enough to work for other file formats as well. We will demonstrate the procedure using 500 `.csv` files. These files have been created using random numbers. Each file contains only 1024 numbers in one column. This method can easily be extended to huge datasets involving thousands of `.csv` files. As the number of files becomes large, we can't load the whole data into memory. So we have to work with chunks of it. Generators help us do just that conveniently. In this post, we will read multiple files using a custom generator.
 
This post is self-sufficient in the sense that readers don't have to download any data from anywhere. Just run the following codes sequentially. First, a folder named "random_data" will be created in current working directory and `.csv` files will be saved in it. Subsequently files will be read from that folder and processed. Just make sure that your current working directory doesn't have an old folder named "random_data". Then run the following codes. Jupyter notebook of this post can be found [here](metion_link).

We will use `Tensorflow 2` to run our deep learning model. `Tensorflow` is very flexible. A given task can be done in different ways in it. The method we will use is not the only one. Readers are encouraged to explore other ways of doing the same. Below is an outline of three different tasks considered in this post.

# Outline:
1. Create 500 `".csv"` files and save it in the folder "random_data" in current directory.
2. Write a generator that reads data from the folder in chunks and preprocesses it.
3. Feed the chunks of data to a CNN model and train it for several epochs.

## 1. Create 500 `.csv` files of random data

As we intend to train a CNN model for classification using our data, we will generate data for 5 different classes. Following is the process that we will follow.
* Each `.csv` file will have one column of data with 1024 entries.
* Each file will be saved using one of the following names (Fault_1, Fault_2, Fault_3, Fault_4, Fault_5). The dataset is balanced, meaning, for each category, we have approximately same number of observations. Data files in "Fault_1" 
category will have names as "Fault_1_001.csv", "Fault_1_002.csv", "Fault_1_003.csv", ..., "Fault_1_100.csv". Similarly for other classes. 

In [1]:
import numpy as np
import os
import glob
np.random.seed(1111)

First create a function that will generate random files. 

In [2]:
def create_random_csv_files(fault_classes, number_of_files_in_each_class):
    os.mkdir("./random_data/")  # Make a directory to save created files.
    for fault_class in fault_classes:
        for i in range(number_of_files_in_each_class):
            data = np.random.rand(1024,)
            file_name = "./random_data/" + eval("fault_class") + "_" + "{0:03}".format(i+1) + ".csv" # This creates file_name
            np.savetxt(eval("file_name"), data, delimiter = ",", header = "V1", comments = "")
        print(str(eval("number_of_files_in_each_class")) + " " + eval("fault_class") + " files"  + " created.")

Now use the function to create 100 files each for five fault types. 

In [3]:
create_random_csv_files(["Fault_1", "Fault_2", "Fault_3", "Fault_4", "Fault_5"], number_of_files_in_each_class = 100)

100 Fault_1 files created.
100 Fault_2 files created.
100 Fault_3 files created.
100 Fault_4 files created.
100 Fault_5 files created.


In [4]:
files = glob.glob("./random_data/*")
files

['./random_data\\Fault_1_001.csv',
 './random_data\\Fault_1_002.csv',
 './random_data\\Fault_1_003.csv',
 './random_data\\Fault_1_004.csv',
 './random_data\\Fault_1_005.csv',
 './random_data\\Fault_1_006.csv',
 './random_data\\Fault_1_007.csv',
 './random_data\\Fault_1_008.csv',
 './random_data\\Fault_1_009.csv',
 './random_data\\Fault_1_010.csv',
 './random_data\\Fault_1_011.csv',
 './random_data\\Fault_1_012.csv',
 './random_data\\Fault_1_013.csv',
 './random_data\\Fault_1_014.csv',
 './random_data\\Fault_1_015.csv',
 './random_data\\Fault_1_016.csv',
 './random_data\\Fault_1_017.csv',
 './random_data\\Fault_1_018.csv',
 './random_data\\Fault_1_019.csv',
 './random_data\\Fault_1_020.csv',
 './random_data\\Fault_1_021.csv',
 './random_data\\Fault_1_022.csv',
 './random_data\\Fault_1_023.csv',
 './random_data\\Fault_1_024.csv',
 './random_data\\Fault_1_025.csv',
 './random_data\\Fault_1_026.csv',
 './random_data\\Fault_1_027.csv',
 './random_data\\Fault_1_028.csv',
 './random_data\\Fau

To extract labels from file name, extract the part of the file name that corresponds to fault type. 

In [5]:
print(files[0])

./random_data\Fault_1_001.csv


In [6]:
print(files[0][14:21])

Fault_1


Now that data has been created, we will go to the next step. That is, define a generator, preprocess the time series like data into a matrix like shape such that a 2-D CNN can ingest it. 

## 2. Write a generator that reads data in chunks and preprocesses it
Generator are similar to functions with one important difference. While functions produce all their outputs at once, generators produce their outputs one by one and that too when asked. `yield` keyword converts a function into a generator. Generators can run for a fixed number of times or indefinitely depending on the loop structure used inside it. For out application, we will use a generator that runs indefinitely.

The following generator takes a list of file names as first input. The second argument is `batch_size`. `batch_size` determines how many files we will process at one go. This is determined by how much memory do we have. If all data can be loaded into memory, there is no need for generators. In case our data size is huge, we can process chunks of it.  

As we will be solving a classification problem, we have to assign labels to each raw data. We will use following labels for convenience.

|Class| Label|
|-----|------|
|Fault_1| 0|
|Fault_2| 1|
|Fault_3| 2|
|Fault_4| 3|
|Fault_5| 4|

The generator will `yield` both data and labels.

In [7]:
import pandas as pd
import re            # To match regular expression for extracting labels

In [8]:
def data_generator(file_list, batch_size = 20):
    i = 0
    while True:
        if i*batch_size >= len(file_list):  # This loop is used to run the generator indefinitely.
            i = 0
            np.random.shuffle(file_list)
        else:
            file_chunk = file_list[i*batch_size:(i+1)*batch_size] 
            data = []
            labels = []
            label_classes = ["Fault_1", "Fault_2", "Fault_3", "Fault_4", "Fault_5"]
            for file in file_chunk:
                temp = pd.read_csv(open(file,'r')) # Change this line to read any other type of file
                data.append(temp.values.reshape(32,32,1)) # Convert column data to matrix like data with one channel
                pattern = "^" + eval("file[14:21]")      # Pattern extracted from file_name
                for j in range(len(label_classes)):
                    if re.match(pattern, label_classes[j]): # Pattern is matched against different label_classes
                        labels.append(j)  
            data = np.asarray(data).reshape(-1,32,32,1)
            labels = np.asarray(labels)
            yield data, labels
            i = i + 1

To read any other file format, inside the generator change the line that reads files. This will enable us to read different file formats, be it `.txt` or `.npz` or any other.

Now we will check whether the generator works as intended or not. We will set `batch_size` to 10. This means that files in chunks of 10 will be read and processed. The list of files from which 10 are chosen can be an ordered file list or shuffled list. In case, the files are not shuffled, use `np.random.shuffle(file_list)` to shuffle files. 

In the demonstration, we will read files from an ordered list. This will help us check any errors in the code. 

In [9]:
generated_data = data_generator(files, batch_size = 10)

In [10]:
num = 0
for data, labels in generated_data:
    print(data.shape, labels.shape)
    print(labels, "<--Labels")  # Just to see the lables
    print()
    num = num + 1
    if num > 5: break

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels



Run the above cell multiple times to observe different labels. Label 1 appears only when all the files corresponding to "Fault_1" have been read. There are 100 files for "Fault_1" and we have set batch_size to 10. In the above cell we are iterating over the generator only 6 times. When number of iterations become greater than 10, we see label 1 and subsequently other labels. This will happen only if our initial file list is not shuffled. If the original list is shuffled, we will get random labels.

Now we will create a `tensorflow dataset` using the generator. `Tensorflow` datasets can conveniently be used to train `tensorflow` models.

A `tensorflow dataset` can be created form numpy arrays or from generators.Here, we will create it using a generator. Use of the previously created generator as it is in `tensorflow datasets` doesn't work (Readers can verify this). This happens because of the inability of regular expression to compare a "string" with a "byte string". "byte strings" are generated by default in tensorflow. As a way around, we will make modifications to the earlier generator and use it with tensorflow datasets. Note that we will only modified three lines. Modified lines are accompanied by commented texts beside it.

In [11]:
import tensorflow as tf

In [12]:
def tf_data_generator(file_list, batch_size = 20):
    i = 0
    while True:
        if i*batch_size >= len(file_list):  
            i = 0
            np.random.shuffle(file_list)
        else:
            file_chunk = file_list[i*batch_size:(i+1)*batch_size] 
            data = []
            labels = []
            label_classes = tf.constant(["Fault_1", "Fault_2", "Fault_3", "Fault_4", "Fault_5"]) # This line has changed.
            for file in file_chunk:
                temp = pd.read_csv(open(file,'r'))
                data.append(temp.values.reshape(32,32,1)) 
                pattern = tf.constant(eval("file[14:21]"))  # This line has changed
                for j in range(len(label_classes)):
                    if re.match(pattern.numpy(), label_classes[j].numpy()):  # This line has changed.
                        labels.append(j)
            data = np.asarray(data).reshape(-1,32,32,1)
            labels = np.asarray(labels)
            yield data, labels
            i = i + 1

Test whether modified generator works or not.

In [13]:
check_data = tf_data_generator(files, batch_size = 10)

In [14]:
num = 0
for data, labels in check_data:
    print(data.shape, labels.shape)
    print(labels, "<--Labels")
    print()
    num = num + 1
    if num > 5: break

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels

(10, 32, 32, 1) (10,)
[0 0 0 0 0 0 0 0 0 0] <--Labels



Note that the new generator created by using a few `tensorflow` commands works just fine as our previous generator. This new generator can now be integrated with a `tensorflow dataset`.

In [15]:
batch_size = 15
dataset = tf.data.Dataset.from_generator(tf_data_generator,args= [files, batch_size],output_types = (tf.float32, tf.float32),
                                                output_shapes = ((None,32,32,1),(None,)))

Check whether `dataset` works or not.

In [16]:
num = 0
for data, labels in dataset:
    print(data.shape, labels.shape)
    print(labels)
    print()
    num = num + 1
    if num > 7: break

(15, 32, 32, 1) (15,)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(15,), dtype=float32)

(15, 32, 32, 1) (15,)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(15,), dtype=float32)

(15, 32, 32, 1) (15,)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(15,), dtype=float32)

(15, 32, 32, 1) (15,)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(15,), dtype=float32)

(15, 32, 32, 1) (15,)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(15,), dtype=float32)

(15, 32, 32, 1) (15,)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(15,), dtype=float32)

(15, 32, 32, 1) (15,)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.], shape=(15,), dtype=float32)

(15, 32, 32, 1) (15,)
tf.Tensor([1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.], shape=(15,), dtype=float32)



This also works fine. Now, we will train a full CNN model using the generator. As is done in every model, we will first shuffle data files. Split the files into train, validation, and test set. Using the `tf_data_generator` create three tensorflow datasets corresponding to train, validation, and test data respectively. Finally, we will create a simple CNN model. Train it using train dataset, see its performance on validation dataset, and obtain prediction using test dataset. Keep in mind that our aim is not to improve performance of the model. As the data are random, don't expect to see good performance. The aim is only to create a pipeline.  

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
train_file_names, test_file_names = train_test_split(files, test_size = 0.2, random_state = 321)

In [19]:
train_file_names, validation_file_names = train_test_split(train_file_names, test_size = 0.15, random_state = 232)

In [20]:
print("Number of train_files:" ,len(train_file_names))
print("Number of validation_files:" ,len(validation_file_names))
print("Number of test_files:" ,len(test_file_names))

Number of train_files: 340
Number of validation_files: 60
Number of test_files: 100


In [21]:
batch_size = 10
train_dataset = tf.data.Dataset.from_generator(tf_data_generator, args = [train_file_names, batch_size], 
                                              output_shapes = ((None,32,32,1),(None,)),
                                              output_types = (tf.float32, tf.float32))

validation_dataset = tf.data.Dataset.from_generator(tf_data_generator, args = [validation_file_names, batch_size],
                                                   output_shapes = ((None,32,32,1),(None,)),
                                                   output_types = (tf.float32, tf.float32))

test_dataset = tf.data.Dataset.from_generator(tf_data_generator, args = [test_file_names, batch_size],
                                             output_shapes = ((None,32,32,1),(None,)),
                                             output_types = (tf.float32, tf.float32))

Now create the model.

In [22]:
from tensorflow.keras import layers

In [23]:
model = tf.keras.Sequential([
    layers.Conv2D(16, 3, activation = "relu", input_shape = (32,32,1)),
    layers.MaxPool2D(2),
    layers.Conv2D(32, 3, activation = "relu"),
    layers.MaxPool2D(2),
    layers.Flatten(),
    layers.Dense(16, activation = "relu"),
    layers.Dense(5, activation = "softmax")
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
_________________________________________________________________
dense (Dense)                (None, 16)                18448     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 8

Compile the model.

In [24]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

Before we fit the model, we have to do one important calculation. Remember that our generators are infinite loops. So if no stopping criteria is given, it will run indefinitely. But we want our model to run for, say, 10 epochs. So our generator should loop over the data files just 10 times and no more. This is achieved by setting the arguments `steps_per_epoch` and `validation_steps` to desired numbers in `model.fit()`. Similarly while evaluating model, we need to set the argument `steps` to a desired number in `model.evaluate()`.

There are 340 files in training set. Batch_size is 10. So if the generator runs 34 times, it will correspond to one epoch. Therefor, we should set `steps_per_epoch` to 34. Similarly, `validation_steps = 6` and in `model.evaluate()`, `steps = 10`.

In [25]:
steps_per_epoch = 34
validation_steps = 6
steps = 10

In [26]:
model.fit(train_dataset, validation_data = validation_dataset, steps_per_epoch = steps_per_epoch,
         validation_steps = validation_steps, epochs = 10)

Train for 34 steps, validate for 6 steps
Epoch 1/10
34/34 [==============================] - 2s 58ms/step - loss: 1.6182 - accuracy: 0.1618 - val_loss: 1.6131 - val_accuracy: 0.2000
Epoch 2/10
34/34 [==============================] - 1s 39ms/step - loss: 1.6086 - accuracy: 0.2029 - val_loss: 1.6177 - val_accuracy: 0.1833
Epoch 3/10
34/34 [==============================] - 1s 39ms/step - loss: 1.6043 - accuracy: 0.2265 - val_loss: 1.6152 - val_accuracy: 0.1000
Epoch 4/10
34/34 [==============================] - 1s 38ms/step - loss: 1.6029 - accuracy: 0.2676 - val_loss: 1.6177 - val_accuracy: 0.1667
Epoch 5/10
34/34 [==============================] - 1s 39ms/step - loss: 1.5976 - accuracy: 0.2294 - val_loss: 1.6301 - val_accuracy: 0.1833
Epoch 6/10
34/34 [==============================] - 1s 40ms/step - loss: 1.5943 - accuracy: 0.2353 - val_loss: 1.6503 - val_accuracy: 0.0833
Epoch 7/10
34/34 [==============================] - 1s 39ms/step - loss: 1.5918 - accuracy: 0.2382 - val_loss: 1.

In [27]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps = 10)

10/10 [==============================] - 0s 32ms/step - loss: 1.6415 - accuracy: 0.1700


In [28]:
print("Test loss: ", test_loss)
print("Test accuracy:", test_accuracy)

Test loss:  1.6414685010910035
Test accuracy: 0.17


As expected, model performs terribly. This brings us to the end of the blog. As we had planned in the beginning, we have created random data files, a generator, and trained a model using that generator. The above code can be tweaked slightly to read any type of files other than `.csv`. And now we can train our model without worrying about the data size. Whether the data size is 10GB or 750GB, our approach will work for both.

As a final note, I want to stress that, this is not the only approach to do the task. As I have mentioned previously, in `Tensorflow`, you can do the same thing in several different ways. The approach I have chosen seemed natural to me. I have neither strived for efficiency nor elegance. If readers have any better idea, I would be happy to know of it.

I hope, this blog will be of help to readers. Please bring any errors or omissions to my notice. 